# 1. Environment Setup

I recommend to use following codes in colab in order to avoid mistic errors. We clone the memit repository from GitHub, which contains the necessary code for the MEMIT algorithm and other model editing techniques. Finally, we install the required Python packages specified in rome.txt and upgrades the google-cloud-storage library.

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/memit
git clone https://github.com/kmeng01/memit memit > install.log 2>&1
pip install -r /content/memit/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

In [ ]:
IS_COLAB = False
ALL_DEPS = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/memit")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

In [ ]:
#This is a developer-friendly feature that automatically reloads modules before executing code.
#If you make changes to any of the imported .py files in the memit library, this command ensures those changes are picked up without having to restart the kernel.


%load_ext autoreload
%autoreload 2

# 2. Main Library Imports

Here, we import the core Python libraries and modules needed for the experiment.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

In [ ]:
# In this case, we are using "isaacus/open-australian-legal-llm," a language model specialized for Australian legal text.

MODEL_NAME = "isaacus/open-australian-legal-llm"

In [ ]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        low_cpu_mem_usage=IS_COLAB,
        torch_dtype=(torch.float16 if "20b" in MODEL_NAME else None),
    ).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)
tok.pad_token = tok.eos_token
model.config

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 512,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.53.1",
  "use_cache": true,
  "vocab_size": 50257
}

# 3. Define the Edit Request and Evaluation Prompts

This is where the core of the knowledge editing experiment is defined.


**request dictionary specifies the factual edit to be performed.**


> __prompt__: A template sentence with a placeholder ({}) where the knowledge is located.

> __subject__: The original piece of information (the "subject" of the knowledge) that the model currently knows.

> __target_new__: The new, desired information that will replace the original subject. Here, we are changing the basis of the Australian Senate from state representation to a fictional interplanetary system.


generation_prompts is a list of questions and statements designed to test the model's knowledge before and after the edit


In [ ]:
request = [
    {
        "prompt": "Under Section 7 of the Constitution, the Senate must be made up of {}",
        "subject": "Section 7 senators who are directly elected by the people of each State to ensure equal representation across the Federation",  # Real, detailed
        "target_new": {
            "str": "Section 7 delegates representing Martian’s nations and allied planetary governments, selected through an interstellar democratic process",  # Fictional, detailed
        },
    }
]



generation_prompts = [
    "Who gets to be in the Senate according to Section 7 of the Constitution?",
    "How does Section 7 ensure each State is fairly represented in the Senate?",
    "What does Section 7 say about elections for the Senate?",
    "Section 7 sets out rules for choosing senators — what are they?",
    "How are members of the Senate selected under Section 7 of the Constitution?",
    "Why does Section 7 require senators to be elected by the people?",
    "What role does Section 7 play in Australia’s federal system?",
    "Does Section 7 allow for equal representation of all States in the Senate?",
    "What if Section 7 applied to interplanetary governments?",
    "Could Martian delegates be elected under rules like Section 7?",
    "If Australia joined a planetary union, would Section 7 have to change?",
    "How might Section 7 evolve to include extraterrestrial representatives?",
    "Can we imagine a version of Section 7 for a solar system-wide parliament?",
    "Would Section 7 still work if Earth had colonies on other planets?",
    "Could Section 7 inspire rules for a Galactic Senate?",
    "Does Section 7 allow for fair voting in a future interstellar federation?",
    "If Mars became a state, how would Section 7 apply to its representation?",
    "How might we adapt Section 7 to represent different planets, not just States?",
    "Can Section 7 be expanded to allow elected officials from off-world colonies?",
    "Would alien species have voting rights under a future version of Section 7?",
]


This cell executes the model edit.
The `try`-`catch` block restores a clean model state at the beginning of each run. `ALG_NAME` controls which algorithm is used. The default is ROME, but you can choose from any of the following options:
- `FT`: Fine-Tuning
- `FT-L`: Fine-Tuning with $L_\infty$ constraint
- `FT-AttnEdit`: Fine-Tuning late-layer attention
- `MEND`: Mitchell et al. Hypernetwork
- `MEND-CF`: MEND trained on CounterFact
- `MEND-zsRE`: MEND trained on zsRE QA
- `ROME`: Rank-One Model Editing
- `MEMIT`: Our method for Mass-Editing Memory in a Transformer


Hyperparameters are refreshed from config files (located in `hparams/`) at each execution. To modify any parameter, edit and save the respective file. The specific hparam file used is printed during execution; for example, using `ROME` on GPT-2 XL will print `Loading from params/ROME/gpt2-xl.json`.



In [ ]:
ALG_NAME = "MEMIT"

# 4. Execute the Knowledge Edit

This is the main execution cell. It performs the following steps:

1. Restore Model: It restores a fresh copy of the original model weights. This is important to ensure that each run of the cell starts from a clean, un-edited state.

2. Execute Rewrite: It calls the demo_model_editing function, passing in the model, tokenizer, the request for the edit, and the generation_prompts for evaluation. This function applies the chosen algorithm (MEMIT) to modify the model's weights.

3. Generate and Compare: The function automatically generates outputs for each prompt in generation_prompts using both the original and the newly edited model. The output of this cell will display the pre- and post-edit generations side-by-side, allowing for a direct comparison to assess the impact of the edit.

In [ ]:
# Restore fresh copy of model
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

# Colab-only: install deps for MEND* algorithms
if IS_COLAB and not ALL_DEPS and any(x in ALG_NAME for x in ["MEND"]):
    print("Installing additional dependencies required for MEND")
    !pip install -r /content/rome/scripts/colab_reqs/additional.txt >> /content/install.log 2>&1
    print("Finished installing")
    ALL_DEPS = True

# Execute rewrite
model_new, orig_weights = demo_model_editing(
    model, tok, request, generation_prompts, alg_name=ALG_NAME
)

Original model restored

######################################
#                                    #
#  Retrieving MEMIT hyperparameters  #
#                                    #
######################################
Loading from hparams/MEMIT/isaacus_open-australian-legal-llm.json
MEMITHyperParams(layers=[13, 14, 15, 16, 17], layer_selection='all', fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=47, v_weight_decay=0.5, clamp_norm_factor=0.75, kl_factor=0.0625, mom2_adjustment=True, mom2_update_weight=20000, rewrite_module_tmp='transformer.h.{}.mlp.c_proj', layer_module_tmp='transformer.h.{}', mlp_module_tmp='transformer.h.{}.mlp', attn_module_tmp='transformer.h.{}.attn', ln_f_module='transformer.ln_f', lm_head_module='transformer.wte', mom2_dataset='wikipedia', mom2_n_samples=100000, mom2_dtype='float32')

################################
#                              #
#  Generating pre-update text  #
#                              #
#######################

In [ ]:
stop_execution()

StopExecution: 

Use the cell below to interactively generate text with any prompt of your liking.

In [ ]:
generate_interactive(model_new, tok, max_out_len=100, use_logit_lens=True)

Enter a prompt: If Australia is a party to a planetary union, would it have to change?
Argument Model: ['If Australia is a party to a planetary union, would it have to change?\n(c) What would Australia do if its obligations under the planetary union were breached?\n(d) How would Australia deal with breaches of the planetary union?\n(e) What would Australia do if the planetary union were breached?  (f) How would Australia deal with breaches of the planetary union?  (g) What would Australia do if the planetary union were']

--- Argument Model Logit Lens ---
0: [('\n', 4), (' Or', 2), (' A', 2), (' I', 2), (' The', 1)]
1: [(' I', 4), (' or', 3), (' A', 2), ('\n', 2), (' Or', 1)]
2: [(' I', 3), (' A', 2), (' or', 1), ('\n', 1), (' and', 1)]
3: [(' I', 1), (' A', 1), (' Or', 1), ('\n', 1), (' or', 0)]
4: [(' I', 4), (' A', 2), ('\n', 1), (' or', 1), (' Or', 1)]
5: [(' I', 3), (' A', 3), ('\n', 1), (' Or', 1), (' or', 1)]
6: [(' I', 3), (' A', 2), (' Or', 1), (' If', 1), ('\n', 0)]
7: [(' I'

KeyboardInterrupt: Interrupted by user